In [2]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('synthetic_credit_card_approval.csv')

# Features and target
X = df.drop('Target', axis=1).values
y = df['Target'].values

# Train-test split
np.random.seed(42)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
train_size = int(0.7 * len(indices))
train_idx, test_idx = indices[:train_size], indices[train_size:]
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# Normalize features
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

# SVM implementation
class SVMScratch:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iter=1000):
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.n_iter = n_iter
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        y_ = np.where(y == 0, -1, 1)
        
        print("Starting training...")
        
        for epoch in range(self.n_iter):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.weights) + self.bias) >= 1
                if condition:
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights)
                else:
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights - np.dot(x_i, y_[idx]))
                    self.bias -= self.learning_rate * y_[idx]
            if (epoch + 1) % 100 == 0 or epoch == 0:
                print(f"Epoch {epoch+1}/{self.n_iter} completed")
        
        print("Training completed.")
    
    def predict(self, X):
        approx = np.dot(X, self.weights) + self.bias
        return np.where(approx >= 0, 1, 0)
    
    def evaluate(self, X, y_true):
        y_pred = self.predict(X)

        accuracy = np.mean(y_pred == y_true)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp + 1e-15)
        recall = tp / (tp + fn + 1e-15)

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print("Confusion Matrix:")
        print(f"TP: {tp}, FP: {fp}")
        print(f"FN: {fn}, TN: {tn}")

# ------------------------
# Train and evaluate
# ------------------------
svm_model = SVMScratch(learning_rate=0.001, lambda_param=0.01, n_iter=1000)
svm_model.fit(X_train, y_train)

print("\nEvaluating on test data:")
svm_model.evaluate(X_test, y_test)


Starting training...
Epoch 1/1000 completed
Epoch 100/1000 completed
Epoch 200/1000 completed
Epoch 300/1000 completed
Epoch 400/1000 completed
Epoch 500/1000 completed
Epoch 600/1000 completed
Epoch 700/1000 completed
Epoch 800/1000 completed
Epoch 900/1000 completed
Epoch 1000/1000 completed
Training completed.

Evaluating on test data:
Accuracy: 0.3854
Precision: 0.3854
Recall: 1.0000
Confusion Matrix:
TP: 57804, FP: 92196
FN: 0, TN: 0
